In [110]:
# old query
from bs4 import BeautifulSoup
from requests_html import HTMLSession

requests = HTMLSession()             # instantiate the HTMLSession Object, this returns executed JS code hence return html is the same as that of the page
                                     # with the requests library, via requests.get(url), that was not the case. The query randomly returns either run or unrun html

<h3> <b> To do </b> <h3>
    <ul>
    <li> add recursive function for Html requests
    <li> querying for all classes
    </ul>

In [86]:
# new html query

# using selenium to scrape dynamic website
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
import os
from bs4 import BeautifulSoup 
from requests_html import HTMLSession
from time import sleep

In [87]:
def load_page(url:str, wait_time, batches):
    '''
    The browser sometimes return different htmls, fortunately there is a small variation in patterns.
    As a workaround, this function tries to reload the page every time a pattern different from the reference html is returned.
    
    Parameters:
    -----------------------
    url :: str
        the url of the parent page
    
    wait_time :: int
        The pause time before the next scroll down is executed, allows the browser to load twelve more hotel cards
        
    batches :: int
        The number of batches loaded (12 per batch), each scroll adds 1 batch of hotel cards
        
    Returns:
    -----------------------
    hotel_cardsBS :: BeautifulSoup Object
        Immediate parent of the hotel cards, the container per se.
    
    '''
    # launch driver load get initial page html, and check if the same as reference html
    driver = webdriver.Chrome(chromedriver_path, options=opts) 
    driver.maximize_window()
    driver.get(url)
    page_init = driver.page_source    
    soup_init = BeautifulSoup(page_init, 'html.parser')
    cardsBS4_init = soup_init.select_one('body .resp-section .inner-section .resp-row .resp-col .main-inner section .h-listing .listings')   
    
    if cardsBS4_init is None:    
        # look at driver.navigate().refresh(), it might be better
        # cardsBS4_init is only none when the html loaded is different from the referrence html, reload.
        driver.quit()
        load_page(url)
    else:
        # hotel
        i = 1
        while i <= batches: 
            # this adds 12 more hotel cards per iteration, if i > 84, the script stops, that is we end up with 1008 (hopefully- net problems) ...
            # ... hotel cards
            script = 'window.scrollTo(0, document.body.scrollHeight,)'
            driver.execute_script(script)                             #scroll down to the bottom
            sleep(wait_time)                                          # gives time for the second scrolldown, adjust according to net speed.
            i+=1
            
        print('done')
        page     = driver.page_source                          # the resulting html after scroll downs
        soup     = BeautifulSoup(page, 'html.parser')
        selector = 'body .resp-section .inner-section .resp-row .resp-col .main-inner section .h-listing .listings'
        hotel_cardsBS = soup.select_one(selector)
        return(hotel_cardsBS)
        

In [88]:
# initial driver arguments
chromedriver_path = os.getcwd() + '\\chromedriver\\chromedriver.exe'
opts = Options()
opts.add_argument(' — headless')

url_q1 = 'https://ph.hotels.com/search.do?resolved-location=COUNTRY%3A10233139%3AUNKNOWN%3AUNKNOWN&destination-'
url_q2 = 'id=10233139&q-destination=Philippines&q-check-in=2021-10-10&q-check-out=2021-10-11&q-rooms=1&q-room-0-adults=2&q-room-0-children=0'
url  = url_q1 + url_q2

hotel_cardsBS = load_page(url, wait_time=10, batches=1)

# working XD

done


# hotel_cards :: BeautifulSoup Object per hotel card in site, with tag li
hotel_cardsBS = soup.select_one('body .resp-section .inner-section .resp-row .resp-col .main-inner section .h-listing .listings')   # BeautifulSoup object of containing all cards
type(hotel_cardsBS)

In [89]:
hotel_cards = hotel_cardsBS.find_all('li', recursive=False) # creates one BS obj per card from hotel card BeautifulSoup
print('we have {0} distinct hotels'.format(len(hotel_cards)))

we have 26 distinct hotels


<h3> <b> Inspecting one of the hotel cards </b> </h3>
Upon inspection the ff were idd.
<li> an a tag with further data given by the attribute href
<li> address: in _2oHhXM contains address with format: specific, town/city, zip, province, Philippines 
<li> amenities: in ?
<li> price: in ? #rate is one day for 2 persons
<li> h ref has it all, must go to href for number of rooms
<blockquote>
    <li> number of rooms in:
    <li> customer reviews in:
    <li> add landmarks, and getting around: in 
</blockquote>

In [90]:
# try getting information from 1 hotel card
# proof of concept
hotel_sample = hotel_cards[1]

In [91]:
hotel_name = hotel_sample['data-title']
hotel_name


'Conrad Manila'

In [9]:
rating = hotel_sample.find('strong',class_='guest-reviews-badge').get_text()
rating_decimal = float(''.join(i for i in filter(None,rating) if (i.isdigit()|(i=='.'))))

In [10]:
amenities     = hotel_sample.select('.hmvt8258-amenities')
amenities_col = [amenity.get_text() for amenity in amenities[0].select('li')]
amenities_col

['Pool', 'Free parking', 'Airport transfer', 'Spa', 'Gym', 'Restaurant']

In [11]:
address = hotel_sample.find('span', class_='address').get_text()
address       # can be split; address.split(', ') returns a list of loc, loc, mmo .... regardless start from last entry
              # ie get the index of Philippines and go back this gives us | address.split(', ')[-1], pa front pero pa back

'Seaside Boulevard, Corner of Coral Way, Mall of Asia Complex, Pasay, Manila, 1300, Philippines'

In [12]:
price = hotel_sample.find('a', class_='price-link').get_text()          # returns the likes of 'P6,221' 
price_int = int(''.join(i for i in filter(None,price) if i.isdigit()))  # filter returns list of all chars in price ['P',',','6','7','8'], loop over each element and 
                                                                        # create a new list satisfying only i.isdigit, join this into a string convert string into int
price_int

6561

In [33]:
href     = hotel_sample.find('a', class_='property-name-link')['href']
href_url = 'https://ph.hotels.com'+ href                                             # this is the link to the individual hotel information

In [32]:
# open the page and get the ff
# number of roms
# customer review
# landmarks

requests = HTMLSession()

page_href = requests.get(href_url) # requests also returns different htmls sometime, do the same recursion used in selenium loading 
soup_href = BeautifulSoup(page_href.content, 'html.parser')

In [34]:
landMarks_q1 = 'body div.resp-section main.inner-section div#property-details '
landMarks_q2 = 'div#flexible-container-bottom div.whats-around-content-landmarks-transport' 
landmarks_transport = soup_href.select_one(landMarks_q1 + landMarks_q2)                        # gets to long if both queries are in one line

landmarks = landmarks_transport.find(class_='whats-around-content landmarks')
landmarks_col = [landmark.get_text() for landmark in landmarks.select('div.landmarks-expandable-wrapper ul.landmark-list li')]

transports = landmarks_transport.find(class_='whats-around-content transport')
transport_categories = [category['class'][0] for category in transports.select('ul')]                 # categegory['class'] returns a list, iregardless the first entry is the actual category

transpo = lambda cat: [station.get_text() for station in transports.select('ul.{0} li'.format(cat))]  # helper function gets the list of all transports under category cat, cat :: str
transports_col = {cat:transpo(cat) for cat in transport_categories}

In [35]:
landmarks_col

['In Barangay 76',
 'SM Mall of Asia - 8 min walk',
 'SMX Convention Center - 3 min walk',
 'Mall of Asia Arena - 6 min walk',
 'World Trade Center Manila - 33 min walk',
 'Cultural Center of the Philippines - 44 min walk',
 "Children's Museum (Museo Pambata) - 4.2 mi / 6.7 km",
 'Rizal Park - 4.4 mi / 7.1 km']

In [36]:
transports_col

{'airport': ['Ninoy Aquino Intl. Airport (MNL) - 20 min drive '],
 'train-station': ['Manila Buenidia Station - 6 min drive ',
  'Manila Vito Cruz Station - 7 min drive ',
  'Manila Paco Station - 7 min drive '],
 'shuttle': ['Airport shuttle (surcharge)']}

In [37]:
hSize_q1 = '#at-a-glance div.cont-wrap '          
hSize_q2 = 'div.fact-sheet-columns div.col-8-24 div.info-box ul li'   # there are two instances of ul li tags in html, select_one always select first instance which is the one we need in our case
hotel_size = soup_href.select_one(hSize_q1 + hSize_q2).get_text()     # this is a bs4 object containing which has a child containing num of rooms
                                                                      # concatenated strings for shorter query in line
    

In [38]:
room_count    = [int(roomCount) for roomCount in hotel_size.split() if roomCount.isdigit()][0]
room_count

347

In [ ]:
'''
Works up to this point

Note:
- Change the link, used different check in and out times since unavailble na yung old check in out query. Was probably due to covid adjustments
  Needs cleaner code
- The individual queries will be removed once code is finished :: was simply following top down formats
- Selenium sometimes load vanilla link, the one with unclear attribute names. Working solution, was to reload quit driver then reload.
- works up to this point added numeral formatting
- comments left
'''

In [41]:
# open another page for the reviews
# starting from the individual hotel card
see_allRevs = soup_href.select_one('div#property-reviews div.see-all-reviews a')
allRevs_url = 'https://ph.hotels.com' + see_allRevs['href']       # to review


In [43]:
reviews_page = requests.get(allRevs_url)
reviews_soup = BeautifulSoup(reviews_page.content, 'html.parser')

review_cards = reviews_soup.select('div.review-card')

# needs another recursion function see first implementation; addresses problem when browser loads different html, keep reloading
# until the right html -- review_cards is not NoneType is loaded.

In [67]:
review_cards
# get only 15

# sample query for one card
# get the name, rating, trip type and the review , delete this later
name = review_cards[0].find(class_='reviewer').get_text()
rating = review_cards[0].find(class_='rating-score').get_text()
trip_type = review_cards[0].find(class_='trip-type-nights').get_text()
comment = review_cards[0].find(class_='expandable-content description').get_text()



# implementation
# the logic of the ff follows from the logic of the ff query :: name = review_cards[0].find(class_='reviewer').get_text()
class_names = ['reviewer', 'rating-score', 'trip-type-nights', 'expandable-content description'] # this are the names of the classes
get_content = lambda card, class_name: card.find(class_= class_name).get_text()
review      = lambda card: {'reviewer': get_content(card, class_names[0]), 
                            'rating': get_content(card, class_names[1]),
                            'trip type': get_content(card, class_names[2]),
                            'comment': get_content(card, class_names[3])
                            }

reviews = [review(card) for card in review_cards[0:15]]   # get formatted card reviews

In [16]:
'''
print(
'''
hotel name                  | 
address                     |
rating                      |
amneties                    |
number of rooms             |
price (2 person/ 1 day)     |
reviews                     |
landmarks                   |
'''
).format(
    hotel_name,
    rating,
    amneties,
    room_count,
    price,
    reviews,
    landmarks)
'''

SyntaxError: invalid syntax (<ipython-input-16-ecc34dba13a1>, line 4)